In [38]:
import json
import pandas as pd
import plotly.express as px

def plot_top5_configuracoes_plotly(json_path, metric='R2_Val'):
    """
    Lê um arquivo JSON de resultados, seleciona as 5 melhores configurações e plota as métricas com Plotly.
    
    Parâmetros:
    -----------
    json_path : str
        Caminho para o arquivo JSON.
    metric : str
        Métrica usada para definir a "melhor" configuração (default: 'R2_Val').
        Pode ser: 'R2_Val', 'RMSE_Val', 'MAE_Val'.
    """
    # Carregar dados JSON
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    df = pd.DataFrame(data)

    # Ordenar os dados (menor é melhor para RMSE e MAE; maior é melhor para R²)
    ascending_order = True if metric in ['RMSE_Val', 'MAE_Val'] else False
    df_sorted = df.sort_values(by=metric, ascending=ascending_order).head(5)

    # Melhor configuração (para destacar)
    best_idx = df_sorted[metric].idxmax() if metric == 'R2_Val' else df_sorted[metric].idxmin()
    df_sorted['Highlight'] = ['Melhor' if idx == best_idx else 'Outros' for idx in df_sorted.index]

    # Label de configuração (transformar dicionário em string curta)
    df_sorted['Config_Label'] = df_sorted['Configuracao'].astype(str)

    # Criar gráfico interativo
    fig = px.bar(
        df_sorted,
        x=metric,
        y='Config_Label',
        orientation='h',
        color='Highlight',
        color_discrete_map={'Melhor': 'lightgreen', 'Outros': 'steelblue'},
        text=df_sorted[metric].round(4),
        title=f"Top 5 Configurações - {df_sorted.iloc[0]['Modelo']} ({metric})"
    )

    # Ajustar layout
    fig.update_traces(textposition='outside')
    fig.update_layout(
        xaxis_title=metric,
        yaxis_title='Configurações',
        showlegend=False,
        height=500,
        width=1500
    )

    fig.show()


In [39]:
import os
import json
import pandas as pd
import plotly.express as px

def plot_melhores_modelos(models_dir, metric='R2_Val'):
    """
    Compara a melhor configuração de cada modelo em um único gráfico.
    
    Parâmetros:
    -----------
    models_dir : str
        Caminho da pasta contendo os arquivos JSON.
    metric : str
        Métrica para avaliar a "melhor" configuração. Pode ser 'R2_Val', 'RMSE_Val' ou 'MAE_Val'.
    """
    melhores = []

    # Iterar sobre todos os arquivos JSON
    for file in os.listdir(models_dir):
        if file.endswith('.json'):
            with open(os.path.join(models_dir, file), 'r') as f:
                data = json.load(f)
                df = pd.DataFrame(data)
                
                # Escolher a melhor configuração
                ascending = True if metric in ['RMSE_Val', 'MAE_Val'] else False
                melhor_config = df.sort_values(by=metric, ascending=ascending).iloc[0]
                
                melhores.append({
                    'Modelo': melhor_config['Modelo'],
                    'Config_Label': str(melhor_config['Configuracao']),
                    metric: melhor_config[metric]
                })

    # Criar DataFrame consolidado
    melhores_df = pd.DataFrame(melhores)

    # Gráfico interativo
    fig = px.bar(
        melhores_df,
        x=metric,
        y='Modelo',
        orientation='h',
        text=melhores_df[metric].round(4),
        color='Modelo',
        hover_data=['Config_Label'],
        title=f'Melhor configuração por modelo ({metric})'
    )

    fig.update_traces(textposition='outside')
    fig.update_layout(
        xaxis_title=metric,
        yaxis_title='Modelos',
        height=500,
        showlegend=False
    )
    
    fig.show()

In [40]:
# Comparar os modelos usando R²
plot_melhores_modelos('data/models_results', metric='R2_Val')

# Comparar os modelos usando RMSE
plot_melhores_modelos('data/models_results', metric='RMSE_Val')

# Comparar os modelos usando MAE
plot_melhores_modelos('data/models_results', metric='MAE_Val')

___
## Escolha do melhor modelo

Com base nos três gráficos fornecidos, no conjunto de validação o modelo XGBoost foi a melhor escolha. A análise dos gráficos de MAE, R2, e RMSE valida essa decisão.

### Análise dos Gráficos
- MAE_Val (Erro Médio Absoluto): Este gráfico (primeira imagem) mostra que o XGBoost tem o menor valor de MAE (0.5783) em comparação com os outros modelos. O MAE mede o erro médio das previsões do modelo, e um valor menor indica que as previsões estão, em média, mais próximas dos valores reais.

- RMSE_Val (Raiz do Erro Quadrático Médio): Similar ao MAE, o RMSE penaliza erros maiores. O XGBoost também apresenta o menor RMSE (0.7539), como visto no terceiro gráfico. Isso confirma que o modelo não apenas tem um erro médio baixo, mas também não comete erros grandes com frequência.

- R2_Val (Coeficiente de Determinação): O gráfico do R2 (segunda imagem) mostra a proporção da variância na variável dependente que é previsível a partir da(s) variável(is) independente(s). O XGBoost tem o maior valor de R2 (0.9943), muito próximo de 1. Um valor de R2 alto indica que o modelo consegue explicar quase toda a variância dos dados, ou seja, suas previsões são extremamente precisas.

___

In [41]:
plot_top5_configuracoes_plotly('data/models_results/XGBoost.json', metric='R2_Val')
plot_top5_configuracoes_plotly('data/models_results/XGBoost.json', metric='RMSE_Val')
plot_top5_configuracoes_plotly('data/models_results/XGBoost.json', metric='MAE_Val')

In [42]:
plot_top5_configuracoes_plotly('data/models_results/Gradient Boosting.json', metric='R2_Val')
plot_top5_configuracoes_plotly('data/models_results/Gradient Boosting.json', metric='RMSE_Val')
plot_top5_configuracoes_plotly('data/models_results/Gradient Boosting.json', metric='MAE_Val')

In [43]:
plot_top5_configuracoes_plotly('data/models_results/KNN.json', metric='R2_Val')
plot_top5_configuracoes_plotly('data/models_results/KNN.json', metric='RMSE_Val')
plot_top5_configuracoes_plotly('data/models_results/KNN.json', metric='MAE_Val')

In [44]:
plot_top5_configuracoes_plotly('data/models_results/MLP.json', metric='R2_Val')
plot_top5_configuracoes_plotly('data/models_results/MLP.json', metric='RMSE_Val')
plot_top5_configuracoes_plotly('data/models_results/MLP.json', metric='MAE_Val')

In [45]:
plot_top5_configuracoes_plotly('data/models_results/LightGBM.json', metric='R2_Val')
plot_top5_configuracoes_plotly('data/models_results/LightGBM.json', metric='RMSE_Val')
plot_top5_configuracoes_plotly('data/models_results/LightGBM.json', metric='MAE_Val')

In [46]:
plot_top5_configuracoes_plotly('data/models_results/RandomForest.json', metric='R2_Val')
plot_top5_configuracoes_plotly('data/models_results/RandomForest.json', metric='RMSE_Val')
plot_top5_configuracoes_plotly('data/models_results/RandomForest.json', metric='MAE_Val')